In [13]:
import pandas as pd
import requests
import bs4 as beautifulsoup
from bs4 import BeautifulSoup
#https://www.nbadraft.net/search-players/ Website we will iterate through

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

base_url = 'http://www.nbadraft.net'


In [45]:
def FetchURL(url):
    #print(url)
    #raise Exception("Stop")
    r = requests.get(url)
    return r.content

In [66]:
def ScrapeProfile():
    pass

url_list = []


#Creating a list of all links to pages where tables of player links are stored

for i in range(10):
    url_link = base_url + "/search-players/"
    if i == 0:
        url_list.append(url_link)
    else: 
        url_link = url_link + "?offset="+str(i)
        url_list.append(url_link)
    
#https://www.nbadraft.net/search-players/ ~ first page
#https://www.nbadraft.net/search-players/?offset=210 ~ all other pages

player_links = []

In [11]:
url_list[:5]

['http://www.nbadraft.net/search-players/',
 'http://www.nbadraft.net/search-players/?offset=1',
 'http://www.nbadraft.net/search-players/?offset=2',
 'http://www.nbadraft.net/search-players/?offset=3',
 'http://www.nbadraft.net/search-players/?offset=4']

In [26]:
"""
for url in soup.find_all("a", {"class_": "odd"}, href=True):
    print(url.text)



for url in soup.find_all('a', {"itemprop": "url"}, href=True):
    print(url.text)
    
"""

'\nfor url in soup.find_all("a", {"class_": "odd"}, href=True):\n    print(url.text)\n\n\n\nfor url in soup.find_all(\'a\', {"itemprop": "url"}, href=True):\n    print(url.text)\n    \n'

In [67]:
def GetPlayerURLs(url):
    #Creates the set of player urls of a single page
    soup = BeautifulSoup(FetchURL(url), "html.parser")

    player_list = []

    for a in soup.find_all(href=True):
        #print("Found the URL:", a["href"], a.text)
        if "https://www.nbadraft.net/players/" in a["href"]:
            player_list.append(a["href"])

    player_set = set(player_list) 
    
    return player_set

def AllPlayerURLs(all_urls=url_list):
    """
    Creates the set of all player urls, from all pages
    """
    complete_set = set()
    
    count_page = 0 
    for url in all_urls:
        print("Scraping page #:", count_page)
        complete_set.update(GetPlayerURLs(url))
        count_page+=1
    
    
    return complete_set
    

AllPlayerURLs()

Scraping page #: 0
Scraping page #: 1
Scraping page #: 2
Scraping page #: 3
Scraping page #: 4
Scraping page #: 5
Scraping page #: 6
Scraping page #: 7
Scraping page #: 8
Scraping page #: 9


{'https://www.nbadraft.net/players/a-j-slaughter/',
 'https://www.nbadraft.net/players/aamir-simms/',
 'https://www.nbadraft.net/players/aaric-murray/',
 'https://www.nbadraft.net/players/aaron-brooks/',
 'https://www.nbadraft.net/players/aaron-bruce/',
 'https://www.nbadraft.net/players/aaron-cosby/',
 'https://www.nbadraft.net/players/aaron-craft/',
 'https://www.nbadraft.net/players/aaron-deloney/',
 'https://www.nbadraft.net/players/aaron-epps/',
 'https://www.nbadraft.net/players/aaron-falzon/',
 'https://www.nbadraft.net/players/aaron-gordon/',
 'https://www.nbadraft.net/players/aaron-gray/',
 'https://www.nbadraft.net/players/aaron-harrison/',
 'https://www.nbadraft.net/players/aaron-henry/',
 'https://www.nbadraft.net/players/aaron-holiday/',
 'https://www.nbadraft.net/players/aaron-jackson/',
 'https://www.nbadraft.net/players/aaron-johnson-cash/',
 'https://www.nbadraft.net/players/aaron-jordan/',
 'https://www.nbadraft.net/players/aaron-menzies/',
 'https://www.nbadraft.net/

In [46]:
soup = BeautifulSoup(url_list[0], 'html.parser')                         
tab=soup.find("table",{"class":"search-table sof-tablepress sticky-enabled"}, href=True)

for a in tab:
    print(a.text)

TypeError: 'NoneType' object is not iterable

In [38]:
soup = BeautifulSoup(url_list[0], 'lxml')
table = soup.find_all('table')
for x in table:
    print(table.text)
    
    
search-table sof-tablepress sticky-enabled



In [29]:
soup

http://www.nbadraft.net/search-players/

In [ ]:
for pages in url_list:
    for player_link in player_links:
        ScrapeProfile()